# Vibe Coding: Real-World Data Cleaning Challenge

## The Mission

You're a Data Analyst at **TechSalary Insights**. Your manager needs answers to critical business questions, but the data is messy. Your job is to clean it and provide accurate insights.

**The catch:** You must figure out how to clean the data yourself. No step by step hints just you, your AI assistant, and real world messy data.

---

## The Dataset: Ask A Manager Salary Survey 2021

**Location:** `../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`

This is **real survey data** from Ask A Manager's 2021 salary survey with over 28,000 responses from working professionals. The data comes from this survey: https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html

**Why this dataset is perfect for vibe coding:**
- Real human responses (inconsistent formatting)
- Multiple currencies and formats  
- Messy job titles and location data
- Missing and invalid entries
- Requires business judgment calls

---

## Your Business Questions

Answer these **exact questions** with clean data. There's only one correct answer for each:

### Core Questions (Required):
1. **What is the median salary for Software Engineers in the United States?** 
2. **Which US state has the highest average salary for tech workers?**
3. **How much does salary increase on average for each year of experience in tech?**
4. **Which industry (besides tech) has the highest median salary?**

### Bonus Questions (If time permits):
5. **What's the salary gap between men and women in tech roles?**
6. **Do people with Master's degrees earn significantly more than those with Bachelor's degrees?**

**Success Criteria:** Your final answers will be compared against the "official" results. Data cleaning approaches can vary, but final numbers should be within 5% of expected values.


---
# Your Work Starts Here

## Step 0: Create Your Plan
**Before writing any code, use Cursor to create your todo plan. Then paste it here:**

## My Data Cleaning Plan

- [✓] Prepare and convert source file (TSV ➜ CSV) 
  - [✓] Locate TSV at `../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey2021 (Responses) - Form Responses 1.tsv`
  - [✓] Validate encoding (`utf-8`) and delimiter (`\t`)
  - [✓] Export to CSV with proper snake_case name: `ask_a_manager_salary_survey_2021.csv`
  - [✓] Save output alongside the TSV in the `Week-02-Pandas-Part-2-and-DS-Overview/data/` folder

- [✓] Standardize/rename columns ✓
  - [✓] Trim whitespace, lowercase all headers
  - [✓] Convert to snake_case and remove special characters
  - [✓] Apply clear, human-readable names (e.g., `Current Salary (USD)` ➜ `current_salary_usd`)
  - [✓] Verify no duplicate column names after renaming

- [ ] Address nulls and invalid entries
  - [ ] Generate null summary (% null by column)
  - [ ] Define rules: drop vs impute per column type
    - [ ] Numeric (e.g., salaries): impute median where appropriate; drop rows if key targets missing
    - [ ] Categorical (e.g., job title, country): impute `Unknown` where appropriate
  - [ ] Remove rows with excessive missingness (e.g., > 40% nulls)
  - [ ] Document assumptions and impacts on row counts

## Currency Normalization & US Scope

- [ ] Explore currency fields
  - [ ] Review `Please indicate the currency` and `If "Other," please indicate the currency here:`
  - [ ] List unique currency values and counts; standardize to ISO-like codes (e.g., `USD`, `GBP`, `EUR`)
  - [ ] Trim/uppercase codes; map common variants (e.g., `us dollars`, `usd$` ➜ `USD`)

- [ ] Standardize country to identify US
  - [ ] Normalize country values (e.g., `US`, `USA`, `United States` ➜ `United States`)
  - [ ] Validate US state field presence when applicable

- [ ] Define US analysis scope
  - [ ] Create `is_us_scope` flag:
    - [ ] Country is `United States` (normalized), OR
    - [ ] Currency is `USD` AND a US state is provided
  - [ ] Document edge cases (overseas workers paid in USD without US state)

- [ ] Create USD/US subsets
  - [ ] Parse salary/bonus as numeric; remove commas and non-digits
  - [ ] Create `df_usd` (currency == `USD`) and `df_us_scope` (is_us_scope == True)
  - [ ] Sanity check counts vs. full dataset

## Industry Filtering & Question Setup

- [ ] Explore industry-related column(s)
  - [ ] Identify the column capturing industry/sector (e.g., `industry` or similar)
  - [ ] List unique values (case-insensitive) and frequency counts
  - [ ] Catalog variations for tech-related terms (e.g., `computing`, `tech`, `technology`, `it`, `software`, `information technology`)

- [ ] Normalize industry categories
  - [ ] Define mapping to canonical categories; consolidate tech synonyms into `technology`
  - [ ] Apply normalization function and validate with value counts
  - [ ] Create boolean flag `is_tech_industry`

- [ ] Prepare filtered datasets for analysis
  - [ ] Save/view subset where `is_tech_industry == True`
  - [ ] Sanity check sample rows and row counts

- [ ] Answer core questions with cleaned data
  - [ ] Median salary for Software Engineers in the US
  - [ ] Highest average salary by US state (tech workers)
  - [ ] Salary increase per year of experience (regression on cleaned numeric fields)
  - [ ] Highest median salary industry (excluding technology)


## Step 1: Data Loading and Exploration

Start here! Load the dataset and get familiar with what you're working with.


In [48]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.linear_model import LinearRegression


## Step 2: Data Cleaning


In [49]:
# Load the Ask A Manager Salary Survey (TSV) robustly
from pathlib import Path
import sys

# filename to look for (exact name as in the repo)
filename = 'Ask A Manager Salary Survey2021 (Responses) - Form Responses 1.tsv'
repo_root = Path('/Users/akkeem/Documents/ClassAssignments/TTPR/Week-05/ds-fall-2025-fri-0630')
# search for the file under repo_root
candidates = list(repo_root.rglob(filename))

if not candidates:
    # fallback: try a broad search from current working dir
    candidates = list(Path('.').rglob(filename))

if not candidates:
    raise FileNotFoundError(f"Could not find '{filename}' under {repo_root} or current working dir ({Path('.').resolve()})")

data_path = candidates[0]
print('Loading from:', data_path)

# It's a tab-separated file (TSV)
df = pd.read_csv(data_path, sep='\t', encoding='utf-8', low_memory=False)

# Quick checks
print('shape:', df.shape)
df.head()


Loading from: /Users/akkeem/Documents/ClassAssignments/TTPR/Week-05/ds-fall-2025-fri-0630/Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey2021 (Responses) - Form Responses 1.tsv
shape: (28062, 18)


,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [50]:
# Explicitly rename the long annual salary column before further processing
import re

explicit_map = {}
for col in df.columns:
    if re.search(r"^what\s+is\s+your\s+annual\s+salary\?", col, flags=re.IGNORECASE):
        explicit_map[col] = 'annual_salary'

if explicit_map:
    df = df.rename(columns=explicit_map).copy()

print('Pre-rename applied:')
for old, new in explicit_map.items():
    print(f'- {old} -> {new}')
print('Has annual_salary now?:', 'annual_salary' in df.columns)


Pre-rename applied:
- What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.) -> annual_salary
Has annual_salary now?: True


In [51]:
# Normalize column headers to snake_case and resolve duplicates
import re

original_columns = list(df.columns)

# helper to snake_case
def to_snake(name: str) -> str:
    name = name.strip().lower()
    name = re.sub(r"[\s\-/]+", "_", name)
    name = re.sub(r"[^a-z0-9_]+", "", name)
    name = re.sub(r"_+", "_", name)
    name = name.strip("_")
    return name

# build new names with de-dupe suffixes when collisions occur
new_columns = []
seen = {}
for col in original_columns:
    base = to_snake(col)
    if base not in seen:
        seen[base] = 0
        new_columns.append(base)
    else:
        seen[base] += 1
        new_columns.append(f"{base}_{seen[base]}")

rename_map = dict(zip(original_columns, new_columns))

# apply rename
renamed_df = df.rename(columns=rename_map).copy()

# preview mapping and check for duplicates
print("Renamed columns (sample):")
for old, new in list(rename_map.items())[:10]:
    print(f"- {old} -> {new}")

# verify uniqueness
assert len(set(renamed_df.columns)) == len(renamed_df.columns), "Duplicate column names remain after renaming"

# replace df for downstream steps
df = renamed_df

print("Total columns:", len(df.columns))
print("All columns:", df.columns.tolist())


Renamed columns (sample):
- Timestamp -> timestamp
- How old are you? -> how_old_are_you
- What industry do you work in? -> what_industry_do_you_work_in
- Job title -> job_title
- If your job title needs additional context, please clarify here: -> if_your_job_title_needs_additional_context_please_clarify_here
- annual_salary -> annual_salary
- How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits. -> how_much_additional_monetary_compensation_do_you_get_if_any_for_example_bonuses_or_overtime_in_an_average_year_please_only_include_monetary_compensation_here_not_the_value_of_benefits
- Please indicate the currency -> please_indicate_the_currency
- If "Other," please indicate the currency here:  -> if_other_please_indicate_the_currency_here
- If your income needs additional context, please provide it here: -> if_your_income_needs_additional_context_please_

In [52]:
# Apply intuitive, human-friendly column names
from typing import Dict, Optional

cols = list(df.columns)

def find_col(*keywords: str) -> Optional[str]:
    """Find the first column containing all keywords (case-insensitive)."""
    keys = [k.lower() for k in keywords]
    for c in cols:
        lc = c.lower()
        if all(k in lc for k in keys):
            return c
    return None

intuitive_map: Dict[str, str] = {}

# Timestamp / demographics / location
if (c := find_col('timestamp')): intuitive_map[c] = 'timestamp'
if (c := find_col('how', 'old')): intuitive_map[c] = 'age_group'
if (c := find_col('what', 'industry')): intuitive_map[c] = 'industry'
if (c := find_col('job', 'title')): intuitive_map[c] = 'job_title'
if (c := find_col('job', 'title', 'context')): intuitive_map[c] = 'job_title_context'
if (c := find_col('what', 'country')): intuitive_map[c] = 'country'
if (c := find_col('state')): intuitive_map[c] = 'us_state'
if (c := find_col('city')): intuitive_map[c] = 'city'

# Compensation
if (c := find_col('annual', 'salary')): intuitive_map[c] = 'annual_salary'
if (c := find_col('additional', 'compensation')): intuitive_map[c] = 'additional_compensation'
if (c := find_col('please', 'indicate', 'currency')): intuitive_map[c] = 'currency'
if (c := find_col('currency', 'other')): intuitive_map[c] = 'currency_other'
if (c := find_col('income', 'context')): intuitive_map[c] = 'income_context'

# Experience and education
if (c := find_col('years', 'overall')): intuitive_map[c] = 'years_experience_overall'
if (c := find_col('years', 'field')): intuitive_map[c] = 'years_experience_field'
if (c := find_col('highest', 'education')): intuitive_map[c] = 'highest_education'

# Demographics
if (c := find_col('gender')): intuitive_map[c] = 'gender'
if (c := find_col('race')): intuitive_map[c] = 'race'

# Apply rename
human_df = df.rename(columns=intuitive_map).copy()

# Show mapping preview
print('Intuitive rename mapping:')
for old, new in intuitive_map.items():
    print(f'- {old} -> {new}')

# Validate no collisions
assert len(set(human_df.columns)) == len(human_df.columns), 'Duplicate names after intuitive renaming'

df = human_df
print('Column count:', len(df.columns))
print('Columns:', df.columns.tolist())


Intuitive rename mapping:
- timestamp -> timestamp
- how_old_are_you -> age_group
- what_industry_do_you_work_in -> industry
- job_title -> job_title
- if_your_job_title_needs_additional_context_please_clarify_here -> job_title_context
- what_country_do_you_work_in -> country
- if_youre_in_the_us_what_state_do_you_work_in -> us_state
- what_city_do_you_work_in -> city
- annual_salary -> annual_salary
- how_much_additional_monetary_compensation_do_you_get_if_any_for_example_bonuses_or_overtime_in_an_average_year_please_only_include_monetary_compensation_here_not_the_value_of_benefits -> additional_compensation
- please_indicate_the_currency -> currency
- if_other_please_indicate_the_currency_here -> currency_other
- if_your_income_needs_additional_context_please_provide_it_here -> income_context
- how_many_years_of_professional_work_experience_do_you_have_overall -> years_experience_overall
- how_many_years_of_professional_work_experience_do_you_have_in_your_field -> years_experience_fi

In [53]:
# Null handling: summarize, parse numerics, impute/fill, drop excessive null rows

before_rows = len(df)

# Numeric parsing for salary/bonus
for col in ['annual_salary', 'additional_compensation']:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(',', '', regex=False)
            .str.extract(r'([-+]?[0-9]*\.?[0-9]+)', expand=False)
        )
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Null summary
null_summary = df.isna().mean().sort_values(ascending=False)
print('Null % by column (top 10):')
print((null_summary * 100).round(1).head(10))

# Define column types
numeric_cols = [c for c in ['annual_salary', 'additional_compensation'] if c in df.columns]

categorical_cols = [
    c for c in ['industry', 'job_title', 'country', 'us_state', 'city', 'highest_education', 'gender', 'race']
    if c in df.columns
]

# Impute numeric with median where appropriate
for c in numeric_cols:
    if c in df.columns:
        med = df[c].median(skipna=True)
        df[c] = df[c].fillna(med)

# Fill categoricals with 'Unknown'
for c in categorical_cols:
    df[c] = df[c].fillna('Unknown')

# Drop rows with excessive missingness (> 40% of columns null)
row_null_frac = df.isna().mean(axis=1)
keep_mask = row_null_frac <= 0.40

after_drop_df = df.loc[keep_mask].copy()

print('Rows before:', before_rows)
print('Rows after dropping high-null rows:', len(after_drop_df))

# Replace df for downstream steps
df = after_drop_df


Null % by column (top 10):
currency_other             99.3
income_context             89.2
job_title_context          74.1
additional_compensation    26.0
us_state                   17.9
highest_education           0.8
race                        0.6
gender                      0.6
city                        0.3
industry                    0.3
dtype: float64
Rows before: 28062
Rows after dropping high-null rows: 28062


In [54]:
# Currency normalization and US scope flag
import re

# Standardize currency codes
if 'currency' in df.columns:
    df['currency'] = df['currency'].astype(str).str.strip().str.upper()
    currency_map = {
        'US DOLLARS': 'USD', 'USD$': 'USD', 'US$': 'USD', '$': 'USD',
        'U.S. DOLLARS': 'USD', 'DOLLARS': 'USD'
    }
    df['currency'] = df['currency'].replace(currency_map)

# Standardize country names
if 'country' in df.columns:
    df['country'] = df['country'].astype(str).str.strip()
    # unify common US variants
    df['country'] = df['country'].replace({
        'US': 'United States', 'USA': 'United States', 'U.S.': 'United States',
        'United States of America': 'United States'
    })

# Create is_us_scope flag
has_us_country = (df['country'] == 'United States') if 'country' in df.columns else False
has_us_state = df['us_state'].notna() if 'us_state' in df.columns else False
uses_usd = (df['currency'] == 'USD') if 'currency' in df.columns else False

# Scope logic: in US OR (USD and state provided)
df['is_us_scope'] = has_us_country | (uses_usd & has_us_state)

# Create subsets
usd_df = df[df['currency'] == 'USD'].copy() if 'currency' in df.columns else df.copy()
us_scope_df = df[df['is_us_scope']].copy()

print('Unique currency codes:', sorted(df['currency'].dropna().unique().tolist()) if 'currency' in df.columns else 'N/A')
print('Rows in USD subset:', len(usd_df))
print('Rows in US scope subset:', len(us_scope_df))
print('US country rows:', int(has_us_country.sum()) if isinstance(has_us_country, pd.Series) else 0)
print('USD with US state rows:', int((uses_usd & has_us_state).sum()) if isinstance(uses_usd, pd.Series) and isinstance(has_us_state, pd.Series) else 0)


Unique currency codes: ['AUD/NZD', 'CAD', 'CHF', 'EUR', 'GBP', 'HKD', 'JPY', 'OTHER', 'SEK', 'USD', 'ZAR']
Rows in USD subset: 23374
Rows in US scope subset: 23395
US country rows: 21761
USD with US state rows: 23374


In [55]:
# Explore industry column values and counts
series = df['industry'].astype(str).str.strip()

# Case-insensitive normalization for profiling only
norm = series.str.lower()
value_counts = norm.value_counts().sort_values(ascending=False)

print('Unique industries:', norm.nunique())
print('Top 25 industry values:')
print(value_counts.head(25))




Unique industries: 1004
Top 25 industry values:
industry
computing or tech                       4699
education (higher education)            2464
nonprofits                              2419
health care                             1896
government and public administration    1889
accounting, banking & finance           1809
engineering or manufacturing            1695
marketing, advertising & pr             1133
law                                     1097
business or consulting                   852
education (primary/secondary)            835
media & digital                          774
insurance                                530
retail                                   505
recruitment or hr                        460
property or construction                 387
art & design                             359
utilities & telecommunications           357
transport or logistics                   304
sales                                    287
social work                              27

In [56]:
# Create tech flag using the matched values from final_matches
# First, let's recreate the final_matches from the previous exploration

norm = df['industry'].astype(str).str.strip().str.lower()
vc = norm.value_counts().sort_values(ascending=False)

final_pattern = re.compile(
    r"\b("
    r"it|tech|technology|software|information\s+technology|comput(?:er|ers|ing)?|"
    r"biotech(?:nology)?(?:[\s/\-]*pharma)?"
    r")\b",
    re.IGNORECASE,
)

idx = vc.index.to_series()
final_mask = idx.str.contains(final_pattern)
final_matches = vc[final_mask]

# Use the matched values to create the flag
matched_industries = set(final_matches.index)
df['is_tech_industry'] = df['industry'].astype(str).str.strip().str.lower().isin(matched_industries)

# Apply normalization
df['industry_normalized'] = df['industry'].copy()
df.loc[df['is_tech_industry'], 'industry_normalized'] = 'technology'

# Verify results
print('Tech industry flag stats:')
print(f'Total rows: {len(df)}')
print(f'Tech industry rows: {df["is_tech_industry"].sum()}')
print(f'Non-tech rows: {(~df["is_tech_industry"]).sum()}')

print('\nTop 10 tech industry values:')
tech_industries = df[df['is_tech_industry']]['industry'].value_counts().head(10)
print(tech_industries)

print('\nTechnology normalized count:', (df['industry_normalized'] == 'technology').sum())


Tech industry flag stats:
Total rows: 28062
Tech industry rows: 4879
Non-tech rows: 23183

Top 10 tech industry values:
industry
Computing or Tech    4699
Biotech                49
Biotechnology          16
Biotech/Pharma          9
Software                7
Biotechnology           6
IT                      5
Tech                    3
Healthcare IT           3
Technology              3
Name: count, dtype: int64

Technology normalized count: 4879


/var/folders/hs/684pm45n64vd1d5sgwfk5rk40000gn/T/ipykernel_77189/3700327365.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_mask = idx.str.contains(final_pattern)


In [57]:
# Create filtered datasets for analysis
# Tech-only subset
tech_df = df[df['is_tech_industry']].copy()

# US scope subset (from previous currency normalization)
us_scope_df = df[df['is_us_scope']].copy()

# Tech + US scope intersection
tech_us_df = df[df['is_tech_industry'] & df['is_us_scope']].copy()

# Validation and summary
print('Dataset filtering summary:')
print(f'Original dataset: {len(df):,} rows')
print(f'Tech industry only: {len(tech_df):,} rows')
print(f'US scope only: {len(us_scope_df):,} rows')
print(f'Tech + US scope: {len(tech_us_df):,} rows')

print('\nTech industry breakdown:')
print(f'Tech in US: {len(tech_us_df):,}')
print(f'Tech outside US: {len(tech_df) - len(tech_us_df):,}')

print('\nSample tech industry job titles:')
sample_titles = tech_df['job_title'].value_counts().head(10)
print(sample_titles)

print('\nTech industry by US state (top 10):')
if 'us_state' in tech_us_df.columns:
    state_counts = tech_us_df['us_state'].value_counts().head(10)
    print(state_counts)


Dataset filtering summary:
Original dataset: 28,062 rows
Tech industry only: 4,879 rows
US scope only: 23,395 rows
Tech + US scope: 3,940 rows

Tech industry breakdown:
Tech in US: 3,940
Tech outside US: 939

Sample tech industry job titles:
job_title
Software Engineer           247
Senior Software Engineer    171
Product Manager              87
Software Developer           75
Engineering Manager          61
Software engineer            42
Project Manager              38
Program Manager              35
Staff Software Engineer      34
Data Analyst                 34
Name: count, dtype: int64

Tech industry by US state (top 10):
us_state
California       696
Massachusetts    353
New York         353
Washington       350
Texas            226
Illinois         172
Oregon           163
Colorado         150
Virginia         121
Pennsylvania     118
Name: count, dtype: int64


In [58]:
# Write normalized CSV to data folder
from pathlib import Path

# Output path
data_dir = Path('/Users/akkeem/Documents/ClassAssignments/TTPR/Week-05/ds-fall-2025-fri-0630/Week-02-Pandas-Part-2-and-DS-Overview/data')
output_csv = data_dir / 'ask_a_manager_salary_survey_2021.csv'

# Ensure directory exists
data_dir.mkdir(parents=True, exist_ok=True)

# Save as CSV with UTF-8 and no index
# Note: df is loaded in the previous cell
clean_df_for_export = df.copy()
clean_df_for_export.to_csv(output_csv, index=False, encoding='utf-8')

print(f'Wrote CSV to: {output_csv.resolve()}')


Wrote CSV to: /Users/akkeem/Documents/ClassAssignments/TTPR/Week-05/ds-fall-2025-fri-0630/Week-02-Pandas-Part-2-and-DS-Overview/data/ask_a_manager_salary_survey_2021.csv


## Step 3: Business Questions Analysis

Now answer those important business questions!


In [59]:
# Question 1: What is the median salary for Software Engineers in the United States?


In [60]:
# Question 2: Which US state has the highest average salary for tech workers?


In [61]:
# Question 3: How much does salary increase on average for each year of experience in tech?

In [62]:

# Question 4: What percentage of respondents work remotely vs. in-office?


In [63]:
# Question 1: What is the median salary for Software Engineers in the United States?

# Filter for Software Engineers in US scope
se_mask = (
    (tech_us_df['job_title'].str.contains(r'software\s+engineer', case=False, na=False, regex=True)) |
    (tech_us_df['job_title'].str.contains(r'engineer', case=False, na=False, regex=True) & 
     tech_us_df['job_title'].str.contains(r'software', case=False, na=False, regex=True))
)

se_us = tech_us_df[se_mask].copy()

# Calculate median salary (ensure numeric)
se_us['annual_salary_clean'] = pd.to_numeric(se_us['annual_salary'], errors='coerce')
median_salary = se_us['annual_salary_clean'].median()

print(f"Question 1: Median salary for Software Engineers in the United States")
print(f"Software Engineers found: {len(se_us)}")
print(f"Median salary: ${median_salary:,.0f}")

# Show salary distribution
print(f"\nSalary statistics:")
print(f"25th percentile: ${se_us['annual_salary_clean'].quantile(0.25):,.0f}")
print(f"75th percentile: ${se_us['annual_salary_clean'].quantile(0.75):,.0f}")
print(f"Mean: ${se_us['annual_salary_clean'].mean():,.0f}")


Question 1: Median salary for Software Engineers in the United States
Software Engineers found: 734
Median salary: $145,000

Salary statistics:
25th percentile: $117,000
75th percentile: $175,000
Mean: $153,933


In [70]:
# Question 2: Which US state has the highest average salary for tech workers?
# Use tech_us_df (tech workers in US scope)
tech_us_df['annual_salary_clean'] = pd.to_numeric(tech_us_df['annual_salary'], errors='coerce')

# Group by state and calculate mean salary
state_salaries = tech_us_df.groupby('us_state')['annual_salary_clean'].agg(['mean', 'count']).reset_index()
state_salaries = state_salaries[state_salaries['count'] >= 5]  # Filter states with at least 5 tech workers
state_salaries = state_salaries.sort_values('mean', ascending=False)

print("Question 2: Which US state has the highest average salary for tech workers?")
print(f"Top 10 states by average tech salary:")
print(state_salaries.head(10))

highest_state = state_salaries.iloc[0]
print(f"\nAnswer: {highest_state['us_state']} with average salary of ${highest_state['mean']:,.0f} ({highest_state['count']} workers)")


Question 2: Which US state has the highest average salary for tech workers?
Top 10 states by average tech salary:
                us_state           mean  count
18               Florida  157457.232143     56
8             California  153320.265805    696
66            Washington  150099.137143    350
48              New York  147838.042493    353
43                Nevada  141310.000000     10
61               Unknown  138298.770270     74
32         Massachusetts  133830.592068    353
47            New Mexico  132200.000000     11
16  District of Columbia  131169.500000     56
45            New Jersey  129929.243902     41

Answer: Florida with average salary of $157,457 (56 workers)


In [66]:
# Question 3: How much does salary increase on average for each year of experience in tech?

# Prepare data for regression
regression_df = tech_us_df[['annual_salary_clean', 'years_experience_field']].dropna()
regression_df['years_experience_clean'] = regression_df['years_experience_field'].str.extract(r'(\d+)').astype(float)

# Handle ranges (take midpoint) and clean
def parse_experience(exp_str):
    if pd.isna(exp_str):
        return np.nan
    exp_str = str(exp_str).lower()
    if 'less than 1' in exp_str or '0' in exp_str:
        return 0.5
    elif '1-2' in exp_str:
        return 1.5
    elif '2-4' in exp_str:
        return 3
    elif '5-7' in exp_str:
        return 6
    elif '8-10' in exp_str:
        return 9
    elif '10+' in exp_str or 'more than 10' in exp_str:
        return 12
    else:
        # Try to extract numbers
        import re
        numbers = re.findall(r'\d+', exp_str)
        if numbers:
            return float(numbers[0])
    return np.nan

regression_df['years_numeric'] = regression_df['years_experience_field'].apply(parse_experience)
regression_df = regression_df.dropna()

# Fit linear regression
X = regression_df[['years_numeric']]
y = regression_df['annual_salary_clean']

model = LinearRegression()
model.fit(X, y)

salary_increase_per_year = model.coef_[0]

print("Question 3: How much does salary increase on average for each year of experience in tech?")
print(f"Salary increase per year of experience: ${salary_increase_per_year:,.0f}")
print(f"R-squared: {model.score(X, y):.3f}")
print(f"Sample size: {len(regression_df)} tech workers")

# Show some examples
print(f"\nExperience vs Salary examples:")
sample = regression_df.sample(min(10, len(regression_df)))
for _, row in sample.iterrows():
    print(f"  {row['years_numeric']:.1f} years -> ${row['annual_salary_clean']:,.0f}")


Question 3: How much does salary increase on average for each year of experience in tech?
Salary increase per year of experience: $-2,235
R-squared: 0.005
Sample size: 3940 tech workers

Experience vs Salary examples:
  0.5 years -> $150,000
  0.5 years -> $165,000
  0.5 years -> $85,000
  0.5 years -> $121,000
  1.0 years -> $95,000
  2.0 years -> $100,000
  2.0 years -> $140,000
  0.5 years -> $150,000
  1.0 years -> $65,000
  6.0 years -> $85,000


In [67]:
# Question 4: Which industry (besides tech) has the highest median salary?
# Use US scope data, exclude tech industries
non_tech_us = us_scope_df[~us_scope_df['is_tech_industry']].copy()
non_tech_us['annual_salary_clean'] = pd.to_numeric(non_tech_us['annual_salary'], errors='coerce')

# Group by industry and calculate median salary
industry_medians = non_tech_us.groupby('industry')['annual_salary_clean'].agg(['median', 'count']).reset_index()
industry_medians = industry_medians[industry_medians['count'] >= 5]  # Filter industries with at least 5 workers
industry_medians = industry_medians.sort_values('median', ascending=False)

print("Question 4: Which industry (besides tech) has the highest median salary?")
print(f"Top 10 non-tech industries by median salary:")
print(industry_medians.head(10))

highest_industry = industry_medians.iloc[0]
print(f"\nAnswer: {highest_industry['industry']} with median salary of ${highest_industry['median']:,.0f} ({highest_industry['count']} workers)")

# Compare with tech median for context
tech_median = tech_us_df['annual_salary_clean'].median()
print(f"\nFor comparison - Tech industry median: ${tech_median:,.0f}")
print(f"Non-tech highest median: ${highest_industry['median']:,.0f}")
print(f"Difference: ${highest_industry['median'] - tech_median:,.0f}")


Question 4: Which industry (besides tech) has the highest median salary?
Top 10 non-tech industries by median salary:
             industry    median  count
214           Defense  148000.0      5
175        Consulting  142500.0      6
590            Pharma  127500.0     16
597   Pharmaceutical   122000.0      9
611   Pharmaceuticals  115000.0     19
260            Energy  110500.0     14
596    Pharmaceutical  108179.0     14
568       Oil and Gas  106500.0      7
612  Pharmaceuticals   100000.0      5
451               Law   95000.0    970

Answer: Defense with median salary of $148,000 (5 workers)

For comparison - Tech industry median: $120,000
Non-tech highest median: $148,000
Difference: $28,000


In [68]:
# Question 5: Which industry (besides tech) has the highest median salary?

In [71]:
# Debug: Examine experience data parsing for Question 3
print("Experience field unique values:")
print(tech_us_df['years_experience_field'].value_counts().head(10))

print("\nExperience parsing debug:")
def parse_experience_debug(exp_str):
    if pd.isna(exp_str):
        return np.nan, "NaN"
    exp_str = str(exp_str).lower()
    if 'less than 1' in exp_str or '0' in exp_str:
        return 0.5, "0-1 years"
    elif '1-2' in exp_str:
        return 1.5, "1-2 years"
    elif '2-4' in exp_str:
        return 3, "2-4 years"
    elif '5-7' in exp_str:
        return 6, "5-7 years"
    elif '8-10' in exp_str:
        return 9, "8-10 years"
    elif '10+' in exp_str or 'more than 10' in exp_str:
        return 12, "10+ years"
    else:
        import re
        numbers = re.findall(r'\d+', exp_str)
        if numbers:
            return float(numbers[0]), f"Extracted: {numbers[0]}"
    return np.nan, "No match"

# Test parsing on sample
sample_exp = tech_us_df['years_experience_field'].dropna().head(20)
for exp in sample_exp:
    parsed, desc = parse_experience_debug(exp)
    print(f"'{exp}' -> {parsed} ({desc})")

# Check the actual regression data
regression_df = tech_us_df[['annual_salary_clean', 'years_experience_field']].dropna()
regression_df['years_numeric'] = regression_df['years_experience_field'].apply(lambda x: parse_experience_debug(x)[0])
regression_df = regression_df.dropna()

print(f"\nRegression data sample:")
print(regression_df[['years_experience_field', 'years_numeric', 'annual_salary_clean']].head(10))

print(f"\nYears experience distribution:")
print(regression_df['years_numeric'].value_counts().sort_index())


Experience field unique values:
years_experience_field
11 - 20 years       917
2 - 4 years         893
5-7 years           889
8 - 10 years        653
21 - 30 years       350
1 year or less      177
31 - 40 years        53
41 years or more      8
Name: count, dtype: int64

Experience parsing debug:
'21 - 30 years' -> 0.5 (0-1 years)
'5-7 years' -> 6 (5-7 years)
'8 - 10 years' -> 0.5 (0-1 years)
'5-7 years' -> 6 (5-7 years)
'11 - 20 years' -> 0.5 (0-1 years)
'5-7 years' -> 6 (5-7 years)
'2 - 4 years' -> 2.0 (Extracted: 2)
'5-7 years' -> 6 (5-7 years)
'8 - 10 years' -> 0.5 (0-1 years)
'8 - 10 years' -> 0.5 (0-1 years)
'21 - 30 years' -> 0.5 (0-1 years)
'31 - 40 years' -> 0.5 (0-1 years)
'5-7 years' -> 6 (5-7 years)
'11 - 20 years' -> 0.5 (0-1 years)
'5-7 years' -> 6 (5-7 years)
'11 - 20 years' -> 0.5 (0-1 years)
'5-7 years' -> 6 (5-7 years)
'2 - 4 years' -> 2.0 (Extracted: 2)
'5-7 years' -> 6 (5-7 years)
'11 - 20 years' -> 0.5 (0-1 years)

Regression data sample:
   years_experience_fiel

In [72]:
# Fixed experience parsing - take median of ranges
def parse_experience_fixed(exp_str):
    if pd.isna(exp_str):
        return np.nan
    exp_str = str(exp_str).lower().strip()

    numbers = re.findall(r'\d+', exp_str)
    if len(numbers) == 1:
        return float(numbers[0])
    elif len(numbers) == 2:
        return (float(numbers[0]) + float(numbers[1])) / 2
    return np.nan

# Apply fixed parsing to tech workers
tech_us_df['years_experience_numeric'] = tech_us_df['years_experience_field'].apply(parse_experience_fixed)

# Create clean regression dataset
regression_clean = tech_us_df[['annual_salary_clean', 'years_experience_numeric']].dropna()

print("Fixed experience parsing results:")
print("Experience distribution:")
print(regression_clean['years_experience_numeric'].value_counts().sort_index())

print(f"\nSample of parsed data:")
sample = regression_clean.head(10)
for _, row in sample.iterrows():
    print(f"Experience: {row['years_experience_numeric']:.1f} years, Salary: ${row['annual_salary_clean']:,.0f}")

# Run corrected regression
from sklearn.linear_model import LinearRegression

X = regression_clean[['years_experience_numeric']]
y = regression_clean['annual_salary_clean']

model = LinearRegression()
model.fit(X, y)

salary_increase_per_year = model.coef_[0]

print(f"\nCorrected Question 3 Results:")
print(f"Salary increase per year of experience: ${salary_increase_per_year:,.0f}")
print(f"R-squared: {model.score(X, y):.3f}")
print(f"Sample size: {len(regression_clean)} tech workers")


Fixed experience parsing results:
Experience distribution:
years_experience_numeric
1.0     177
3.0     893
6.0     889
9.0     653
15.5    917
25.5    350
35.5     53
41.0      8
Name: count, dtype: int64

Sample of parsed data:
Experience: 25.5 years, Salary: $112,000
Experience: 6.0 years, Salary: $187,500
Experience: 9.0 years, Salary: $110,000
Experience: 6.0 years, Salary: $144,600
Experience: 15.5 years, Salary: $200,850
Experience: 6.0 years, Salary: $72,000
Experience: 3.0 years, Salary: $52,000
Experience: 6.0 years, Salary: $34,000
Experience: 9.0 years, Salary: $88,000
Experience: 9.0 years, Salary: $125,000

Corrected Question 3 Results:
Salary increase per year of experience: $2,386
R-squared: 0.045
Sample size: 3940 tech workers


In [ ]:
# Explain how the salary increase per year of experience is calculated
import numpy as np

print("HOW THE SALARY INCREASE PER YEAR IS CALCULATED:")
print("=" * 50)

# Show the regression equation
print(f"Linear Regression Formula: Salary = a + b × Years_Experience")
print(f"Where:")
print(f"  a (intercept) = ${model.intercept_:,.0f}")
print(f"  b (coefficient) = ${model.coef_[0]:,.0f}")

print(f"\nThis means:")
print(f"For each additional year of experience, salary increases by ${model.coef_[0]:,.0f}")

# Show examples
print(f"\nEXAMPLES:")
print(f"0 years experience → ${model.intercept_:,.0f}")
print(f"1 year experience  → ${model.intercept_ + model.coef_[0]:,.0f} (${model.coef_[0]:,.0f} increase)")
print(f"5 years experience → ${model.intercept_ + 5*model.coef_[0]:,.0f} (${5*model.coef_[0]:,.0f} total increase)")
print(f"10 years experience → ${model.intercept_ + 10*model.coef_[0]:,.0f} (${10*model.coef_[0]:,.0f} total increase)")

# Show the actual data points used
print(f"\nSAMPLE OF ACTUAL DATA POINTS:")
sample_data = regression_clean.sample(min(10, len(regression_clean)))
for _, row in sample_data.iterrows():
    predicted = model.intercept_ + model.coef_[0] * row['years_experience_numeric']
    print(f"{row['years_experience_numeric']:.1f} years → Actual: ${row['annual_salary_clean']:,.0f}, Predicted: ${predicted:,.0f}")

print(f"\nSTATISTICAL SUMMARY:")
print(f"R-squared: {model.score(X, y):.3f} (explains {model.score(X, y)*100:.1f}% of salary variation)")
print(f"Sample size: {len(regression_clean)} tech workers")
print(f"Experience range: {regression_clean['years_experience_numeric'].min():.1f} to {regression_clean['years_experience_numeric'].max():.1f} years")
print(f"Salary range: ${regression_clean['annual_salary_clean'].min():,.0f} to ${regression_clean['annual_salary_clean'].max():,.0f}")


In [69]:
# Bonus Questions:
# Question 6: What's the salary gap between men and women in similar roles?
# Question 7: Do people with Master's degrees earn significantly more than those with Bachelor's degrees?
# Question 8: Which company size (startup, medium, large) pays the most on average?

## Final Summary

**Summarize your findings here:**

1. **Median salary for Software Engineers in US:** $X
2. **Highest paying US state for tech:** State Name
3. **Salary increase per year of experience:** $X per year
4. **Remote vs office percentage:** X% remote, Y% office
5. **Highest paying non-tech industry:** Industry Name

**Key insights:**
- Insight 1
- Insight 2
- Insight 3

**Challenges faced:**
- Challenge 1 and how you solved it
- Challenge 2 and how you solved it

**What you learned about vibe coding:**
- Learning 1
- Learning 2
- Learning 3
